In [ ]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn import model_selection
import xgboost as xgb
from sklearn import preprocessing
from sklearn import metrics
import random

In [ ]:
def create_folds(df, split):
    #create a new column
    df['kfold'] = -1

    #Shuffle
    df = df.sample(frac =1).reset_index(drop= True)

    # target values
    y= df.target.values
    
    #initiate fold class 
    kf = model_selection.StratifiedKFold(n_splits= split)

    #create splits
    for fold_num, (r_old, r_new) in enumerate(kf.split(X=df, y= y)):
      df.loc[r_new, 'kfold'] = int(fold_num)
    
    return pd.DataFrame(df)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/train.csv')

In [ ]:
df

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,599995,0.0,1.0,0.0,T,N,Red,Polygon,Axolotl,India,Theremin,014770cf0,da5014b01,a7059911d,158183c63,015c63324,3.0,Novice,Freezing,a,R,GZ,5.0,NaN,0
599996,599996,1.0,0.0,0.0,T,Y,Blue,Polygon,Dog,Costa Rica,Oboe,NaN,2023ed4ed,83bdea3a5,e9fde8fa8,a02ae6a63,2.0,Novice,Boiling Hot,n,N,sf,NaN,3.0,0
599997,599997,0.0,0.0,0.0,F,Y,Red,Circle,Axolotl,Russia,Theremin,c7dc5d460,5d7d341ac,114b1dbf3,cccbca824,40f9610c1,2.0,Contributor,Freezing,n,H,MV,7.0,5.0,0
599998,599998,1.0,1.0,0.0,F,Y,NaN,Polygon,Axolotl,NaN,Piano,4d7780407,209e1054e,fba315672,4164322bd,c1a8374a0,1.0,Master,Warm,m,X,Ey,1.0,5.0,0


In [ ]:
df = create_folds(df,5)

In [ ]:
# features of data frame
features = [f for f in df.columns if f not in ('id','target','kfold')]

In [ ]:
features

['bin_0',
 'bin_1',
 'bin_2',
 'bin_3',
 'bin_4',
 'nom_0',
 'nom_1',
 'nom_2',
 'nom_3',
 'nom_4',
 'nom_5',
 'nom_6',
 'nom_7',
 'nom_8',
 'nom_9',
 'ord_0',
 'ord_1',
 'ord_2',
 'ord_3',
 'ord_4',
 'ord_5',
 'day',
 'month']

In [ ]:
#handle missing values
for column in features:
    df.loc[:,column] = df[column].astype(str).fillna('NONE')

In [ ]:
df.isnull().sum()

id        0
bin_0     0
bin_1     0
bin_2     0
bin_3     0
bin_4     0
nom_0     0
nom_1     0
nom_2     0
nom_3     0
nom_4     0
nom_5     0
nom_6     0
nom_7     0
nom_8     0
nom_9     0
ord_0     0
ord_1     0
ord_2     0
ord_3     0
ord_4     0
ord_5     0
day       0
month     0
target    0
kfold     0
dtype: int64

In [ ]:
# Label encoding the features
for column in features:
    lbl = preprocessing.LabelEncoder()
    df.loc[:,column] = lbl.fit_transform(df[column])

In [ ]:
df

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,76040,0,0,0,1,1,0,5,0,5,3,708,723,57,80,808,1,4,1,1,3,184,6,10,1,0
1,453537,2,0,1,1,0,1,5,0,6,3,192,768,141,155,55,2,4,1,13,17,12,5,8,0,0
2,398770,0,0,0,1,1,2,5,3,2,4,1037,1411,39,55,973,2,1,2,14,13,69,4,4,0,0
3,219114,0,0,1,0,1,0,0,4,4,3,98,625,112,115,1863,2,5,4,10,23,46,4,4,0,0
4,165113,0,0,1,1,0,2,1,2,2,3,1057,476,5,158,2064,2,0,5,4,19,117,5,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,538213,0,0,0,0,2,3,0,4,5,1,992,108,132,115,582,2,0,1,14,4,178,2,9,1,4
599996,436860,0,0,1,0,0,2,1,6,2,4,807,1321,9,167,859,0,1,5,0,17,98,2,9,0,4
599997,191294,1,0,0,0,0,2,0,3,4,3,1068,1431,10,126,1716,1,2,2,3,13,140,6,6,0,4
599998,35695,0,0,0,1,1,2,5,1,2,0,582,1323,10,22,575,2,5,6,3,5,21,6,5,0,4


In [ ]:
fold = [0,1,2,3,4]
f= random.choice(fold)
#use the folds
df_train = df[df.kfold !=f].reset_index(drop=True)
df_valid = df[df.kfold ==f].reset_index(drop=True)

In [ ]:
df_train.kfold.value_counts()

4    120000
2    120000
1    120000
0    120000
Name: kfold, dtype: int64

In [ ]:
df_valid.kfold.value_counts()

3    120000
Name: kfold, dtype: int64

In [ ]:
#getting the training variables
x_train = df_train[features].values
x_valid = df_valid[features].values
y_train = df_train.target.values
y_valid = df_valid.target.values

array([[  0,   0,   0, ..., 184,   6,  10],
       [  2,   0,   1, ...,  12,   5,   8],
       [  0,   0,   0, ...,  69,   4,   4],
       ...,
       [  1,   0,   0, ..., 140,   6,   6],
       [  0,   0,   0, ...,  21,   6,   5],
       [  0,   0,   0, ...,  14,   6,   8]])

In [ ]:
# initialize xgboost model
model = xgb.XGBClassifier( n_jobs = -1, max_depth = 5, n_estimators =200)
model.fit (x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = model.predict_proba(x_valid)[:,1]

In [ ]:
pd.DataFrame(y_pred)

,0,1
0,0.914154,0.085846
1,0.837928,0.162072
2,0.952817,0.047182
3,0.901929,0.098071
4,0.745751,0.254249
...,...,...
119995,0.643727,0.356273
119996,0.699730,0.300270
119997,0.858019,0.141981
119998,0.796180,0.203820


In [ ]:
auc = metrics.roc_auc_score(y_valid, y_pred)
auc

0.7627213315473611